In [ ]:
#import torch
#print(torch.__version__)
!pip uninstall torch -y
!pip uninstall torchvision -y
!pip uninstall torchaudio -y

#!nvcc --version
!pip install torch==1.9.0 torchvision==0.10.0 torchaudio==0.9.0

!pip install timm==0.4.9 einops

!pip install apex

!pip install mmdet

!pip install matplotlib

!pip install mmcv-full==1.3.9 -f https://download.openmmlab.com/mmcv/dist/cu111/torch1.9.0/index.html

!git clone https://github.com/ViTAE-Transformer/ViTPose.git
!pip install -v -e ./ViTPose

Found existing installation: torch 1.11.0+cu113
Uninstalling torch-1.11.0+cu113:
  Successfully uninstalled torch-1.11.0+cu113
Found existing installation: torchvision 0.12.0+cu113
Uninstalling torchvision-0.12.0+cu113:
  Successfully uninstalled torchvision-0.12.0+cu113
Found existing installation: torchaudio 0.11.0+cu113
Uninstalling torchaudio-0.11.0+cu113:
  Successfully uninstalled torchaudio-0.11.0+cu113
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████▏           | 522.7 MB 1.2 MB/s eta 0:04:13

In [ ]:
# Results from this repo on MS COCO val set (multi-task training)
# MODEL B
!wget -O model_b.pth https://fd0rfg.sn.files.1drv.com/y4mjxZwPM-eGNFmXLsf91PehujeQMrrF_3u34fFj5aaZyJVLka85mkdL8CXELnZUv5dINCzwWk_gtuxjs5DSluasUcio_U1DRqNAnMFeaACjUuYq65nIV7UBzeutRE1OE2HYfkD79eXHaD-VR0r8txxAQot2TQOnafJgBfPQSxgMyak93Gt8_V7fr4WZFJbFy_fasyBRBl1OYh_jOr--167fQ

In [ ]:
!cd ViTPose && mkdir data
!cd ViTPose/data && mkdir coco
#!unzip annotations_trainval2017.zip -d ViTPose/data/coco

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import copy
import json
import os

import numpy as np
from mmpose.apis import (inference_bottom_up_pose_model,
                         inference_top_down_pose_model, init_pose_model,
                         process_mmdet_results, vis_pose_result)
from mmpose.datasets import DatasetInfo

In [ ]:
def init_model():
  return init_pose_model(
      '/content/ViTPose/configs/body/2d_kpt_sview_rgb_img/topdown_heatmap/'
      'coco/ViTPose_base_coco_256x192.py',
      'model_b.pth',
      device='cpu')

def init_dataset_info(pose_model):
  dataset_info = DatasetInfo(pose_model.cfg.data['test'].get(
      'dataset_info', None))
    

def vis_top_down(pose_model, dataset_info, person_result, image_path):
  # # test a single image, with a list of bboxes.
  pose_results = get_pose(
      pose_model, dataset_info, person_result, image_path)
  
  # show the results
  return vis_pose_result(
      pose_model, image_path, pose_results, dataset_info=dataset_info)

def get_pose(pose_model, dataset_info, person_result, image_path):
  # # test a single image, with a list of bboxes.
  pose_results, _ = inference_top_down_pose_model(
      pose_model,
      image_path,
      person_result,
      format='xyxy',
      dataset_info=dataset_info)
  return pose_results

In [ ]:
from google.colab import files
from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
with open('/content/drive/MyDrive/AdvancedComputerVision/annotation/annotation_image_info.json') as f:
  annotations = json.load(f)

pose_model = init_model()
dataset_info = init_dataset_info(pose_model)

Use load_from_local loader


In [ ]:
!mkdir images_with_detection

In [ ]:
class ImageDetections():
  def __init__(self, image_id):
    self.image_id = image_id  
    self.people_poses = []

  def add_pose(self, pose):
    self.people_poses.append(pose)

  def toJSON(self):
    return json.dumps(self, default=lambda o: o.__dict__,
                     sort_keys=True, indent=4)
    
class PersonDetection():
  def __init__(self, left_eye, right_eye, left_ear, right_ear,  left_shoulder, 
               right_shoulder, left_elbow, right_elbow, left_wrist, right_wrist, 
               left_hip, right_hip, left_knee, right_knee, left_ankle, right_ankle):

    self.left_eye = left_eye.tolist()
    self.right_eye = right_eye.tolist()
    self.left_ear = left_ear.tolist()
    self.right_ear = right_ear.tolist()
    self.left_shoulder = left_shoulder.tolist()
    self.right_shoulder = right_shoulder.tolist()
    self.left_elbow = left_elbow.tolist()
    self.right_elbow = right_elbow.tolist()
    self.left_wrist = left_wrist.tolist()
    self.right_wrist = right_wrist.tolist()
    self.left_hip = left_hip.tolist()
    self.right_hip = right_hip.tolist()
    self.left_knee = left_knee.tolist()
    self.right_knee = right_knee.tolist()
    self.left_ankle = left_ankle.tolist()
    self.right_ankle = right_ankle.tolist()

  # def __str__(self):
  #   return    "\nleft_eye:\t " + str(self.left_eye) + \
  #                 "\nright_eye:\t " + str(self.right_eye) + \
  #                 "\nleft_ear:\t " + str(self.left_ear) + \
  #                 "\nright_ear:\t " + str(self.right_eay) + \
  #                 "\nleft_shoulder:\t " + str(self.left_shoulder) + \
  #                 "\nright_shoulder:\t " + str(self.right_shoulder) + \
  #                 "\nleft_elbow:\t " + str(self.left_elbow) + \
  #                 "\nright_elbow:\t " + str(self.right_elbow) + \
  #                 "\nleft_wrist:\t " + str(self.left_wrist) + \
  #                 "\nright_wrist:\t " + str(self.right_wrist) + \
  #                 "\nleft_hip:\t " + str(self.left_hip) + \
  #                 "\nright_hip:\t " + str(self.right_hip) + \
  #                 "\nleft_knee:\t " + str(self.left_knee) + \
  #                 "\nright_knee:\t " + str(self.right_knee) + \
  #                 "\nleft_ankle:\t " + str(self.left_ankle) + \
  #                 "\nright_ankle:\t " + str(self.right_ankle)

In [ ]:
#Creates a new detection object (check the config for the meaning of the indexes)
def create_detection(detection):
  return PersonDetection(
      detection[0],
      detection[1],
      detection[2],
      detection[3],
      detection[4],
      detection[5],
      detection[6],
      detection[7],
      detection[8],
      detection[9],
      detection[10],
      detection[11],
      detection[12],
      detection[13],
      detection[14],
      detection[15]
      )

In [ ]:
#@title Configuration for the joints that we use { display-mode: "form" }
# Copied from https://github.com/ViTAE-Transformer/ViTPose/blob/33f49cf1f24077ec175cef0990bd25d2746c4898/configs/_base_/datasets/coco_wholebody.py
# It contains the indexes for the configuration for the joints
# For example: the left eye can be saved with the id=1 and this configuration
# can be used to understand what the id is referring to
dataset_info = dict(
    keypoint_info={
        0:
        dict(name='nose', id=0, color=[51, 153, 255], type='upper', swap=''),
        1:
        dict(
            name='left_eye',
            id=1,
            color=[51, 153, 255],
            type='upper',
            swap='right_eye'),
        2:
        dict(
            name='right_eye',
            id=2,
            color=[51, 153, 255],
            type='upper',
            swap='left_eye'),
        3:
        dict(
            name='left_ear',
            id=3,
            color=[51, 153, 255],
            type='upper',
            swap='right_ear'),
        4:
        dict(
            name='right_ear',
            id=4,
            color=[51, 153, 255],
            type='upper',
            swap='left_ear'),
        5:
        dict(
            name='left_shoulder',
            id=5,
            color=[0, 255, 0],
            type='upper',
            swap='right_shoulder'),
        6:
        dict(
            name='right_shoulder',
            id=6,
            color=[255, 128, 0],
            type='upper',
            swap='left_shoulder'),
        7:
        dict(
            name='left_elbow',
            id=7,
            color=[0, 255, 0],
            type='upper',
            swap='right_elbow'),
        8:
        dict(
            name='right_elbow',
            id=8,
            color=[255, 128, 0],
            type='upper',
            swap='left_elbow'),
        9:
        dict(
            name='left_wrist',
            id=9,
            color=[0, 255, 0],
            type='upper',
            swap='right_wrist'),
        10:
        dict(
            name='right_wrist',
            id=10,
            color=[255, 128, 0],
            type='upper',
            swap='left_wrist'),
        11:
        dict(
            name='left_hip',
            id=11,
            color=[0, 255, 0],
            type='lower',
            swap='right_hip'),
        12:
        dict(
            name='right_hip',
            id=12,
            color=[255, 128, 0],
            type='lower',
            swap='left_hip'),
        13:
        dict(
            name='left_knee',
            id=13,
            color=[0, 255, 0],
            type='lower',
            swap='right_knee'),
        14:
        dict(
            name='right_knee',
            id=14,
            color=[255, 128, 0],
            type='lower',
            swap='left_knee'),
        15:
        dict(
            name='left_ankle',
            id=15,
            color=[0, 255, 0],
            type='lower',
            swap='right_ankle'),
        16:
        dict(
            name='right_ankle',
            id=16,
            color=[255, 128, 0],
            type='lower',
            swap='left_ankle'),
    })

In [ ]:
# Saves images with the body pose detection
# def save_body_pose_images():
#   directory = '/content/drive/MyDrive/AdvancedComputerVision/image/'
#   boxes = []
#   cont = 0

#   for image_name in sorted(os.listdir(directory)):
#     print(int(image_name[:-4]))
#     for box in annotations[int(image_name[:-4])]['bbox']:
#       boxes.append({'bbox': box})
#       cont = cont + 1
  
#   figure = vis_top_down(pose_model, dataset_info, boxes, directory+image_name)
#   im = Image.fromarray(figure)
#   im.save("images_with_detection/"+image_name)
#   boxes = []

In [ ]:
# Estimates poses and creates Detection objects for each pose
def detect_poses():
  directory = '/content/drive/MyDrive/AdvancedComputerVision/image/'
  boxes = []
  list_poses = []
  image_detection_list = []
  cont = 0

  # Loop images in the folder
  for image_name in sorted(os.listdir(directory)):
    image_id = int(image_name[:-4])
    print(image_id+1)

    image_detection = ImageDetections(image_id)
    # Create an object with the file and the list of poses
    image_detection_list.append(image_detection)

    # Obtain annotations for the given image
    for box in annotations[image_id]['bbox']:
      boxes.append({'bbox': box})

    # Obtain poses for each bbox
    obtained_poses = get_pose(pose_model, dataset_info, boxes, directory+image_name)
    
    # For each pose create a detection object
    for single_pose in obtained_poses:
      image_detection.add_pose(
          create_detection(
              single_pose['keypoints'][:,:-1]))
    
    boxes = []
    # Break the loop (just for testing)
    cont = cont + 1
    if cont==10:
      return image_detection_list

In [ ]:
image_detection_list = []
directory = '/content/drive/MyDrive/AdvancedComputerVision/image/'

# Estimates poses and creates Detection objects for each pose
def detect_poses(image_name):
  boxes = []
  list_poses = []
  cont = 0

  image_id = int(image_name[:-4])
  print(image_id+1)

  image_detection = ImageDetections(image_id)
  # Create an object with the file and the list of poses
  image_detection_list.append(image_detection)

  # Obtain annotations for the given image
  for box in annotations[image_id]['bbox']:
    boxes.append({'bbox': box})

  # Obtain poses for each bbox
  obtained_poses = get_pose(pose_model, dataset_info, boxes, directory+image_name)
  
  # For each pose create a detection object
  for single_pose in obtained_poses:
    image_detection.add_pose(
        create_detection(
            single_pose['keypoints'][:,:-1]))


In [ ]:
# #Parallelise the process, so to compute the same for multiple images
# from multiprocessing import Pool

# directory = '/content/drive/MyDrive/AdvancedComputerVision/image/'
# list_images = sorted(os.listdir(directory))

# pool = Pool(os.cpu_count())
# pool.map(detect_poses, list_images)
# pool.close()

In [ ]:
# def test_method():
#   boxes = []
  
#   for box in annotations[1]['bbox']:
#     boxes.append({'bbox': box})
  
#   return get_pose(pose_model, dataset_info, boxes, '00001.jpg')

# image_1 = test_method()

In [ ]:
# The third column (index 2) represents the score (a sort of accuracy)
# print(image_1[0])
# print(image_1[0]['keypoints'])
# print(image_1[0]['keypoints'][:,:-1][0])

In [ ]:
list_poses = detect_poses()

# Serialization
with open("serialized_poses.json", "w") as outfile:
    json.dump(list_poses, outfile, default=lambda o: o.__dict__ )

1
2
3
4
5
6
7
8
9
10


In [ ]:
#list_poses[0].people_poses[0].__dict__

In [ ]:
#list_poses[0].toJSON()

In [ ]:
#json.dumps(list_poses, default=lambda o: o.__dict__)